### This resource is from internet
https://www.apsl.net/blog/2017/07/18/using-linear-discriminant-analysis-lda-data-explore-step-step/

In [254]:
import pandas as pd
import numpy as np

feature_dict = {i:label for i,label in zip(
                range(4),
                  ('sepal length in cm',
                  'sepal width in cm',
                  'petal length in cm',
                  'petal width in cm', ))}

import pandas as pd

df = pd.io.parsers.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
    header=None,
    sep=',',
    )
df.columns = [l for i,l in sorted(feature_dict.items())] + ['class label']
df.dropna(how="all", inplace=True) # to drop the empty line at file-end

df.head()

,sepal length in cm,sepal width in cm,petal length in cm,petal width in cm,class label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [251]:
%matplotlib inline

from sklearn.preprocessing import LabelEncoder

my_df =df.loc[:, :].values
X = my_df[:, 0:4]
y = my_df[:, 4]

enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y) + 1

label_dict = {1: 'Setosa', 2: 'Versicolor', 3:'Virginica'}

print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3]


In [253]:
np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl], axis=0))
    print('Mean Vector class %s: %s\n' %(cl, mean_vectors[cl-1]))

Mean Vector class 1: [5.005999999999999 3.4180000000000006 1.464 0.2439999999999999]

Mean Vector class 2: [5.936 2.7700000000000005 4.26 1.3259999999999998]

Mean Vector class 3: [6.587999999999998 2.9739999999999998 5.552 2.026]



In [227]:
S_W = np.zeros((4,4))
for cl,mv in zip(range(1,4), mean_vectors):
    class_sc_mat = np.zeros((4,4))                  # scatter matrix for every class
    for row in X[y == cl]:
        row, mv = row.reshape(4,1), mv.reshape(4,1) # make column vectors
        class_sc_mat = class_sc_mat + (row-mv).dot((row-mv).T)
    S_W = S_W + class_sc_mat                             # sum class scatter matrices
print('within-class Scatter Matrix:\n', S_W)

within-class Scatter Matrix:
 [[38.956199999999995 13.682999999999996 24.614000000000004
  5.6556000000000015]
 [13.682999999999996 17.035000000000004 8.12 4.9132]
 [24.614000000000004 8.12 27.220000000000017 6.2536000000000005]
 [5.6556000000000015 4.9132 6.2536000000000005 6.175599999999998]]


In [247]:
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((4,4))
for i,mean_vec in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4,1) # make column vector
    overall_mean = overall_mean.reshape(4,1) # make column vector
    S_B =S_B + n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)

50
50
50
between-class Scatter Matrix:
 [[63.21213333333327 -19.534000000000034 165.16466666666656
  71.36306666666663]
 [-19.534000000000034 10.97760000000001 -56.05520000000008
  -22.492400000000032]
 [165.16466666666656 -56.05520000000008 436.6437333333333
  186.90813333333332]
 [71.36306666666663 -22.492400000000032 186.90813333333332
  80.60413333333332]]


In [230]:
S_W = S_W.astype('float64')
S_B = S_B.astype('float64')

eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(4,1)   
    print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
    print('Eigenvalue {:}: {:.2e}'.format(i+1, eig_vals[i].real))



Eigenvector 1: 
[[-0.2049]
 [-0.3871]
 [ 0.5465]
 [ 0.7138]]
Eigenvalue 1: 3.23e+01

Eigenvector 2: 
[[-0.009 ]
 [-0.589 ]
 [ 0.2543]
 [-0.767 ]]
Eigenvalue 2: 2.78e-01

Eigenvector 3: 
[[ 0.6104]
 [-0.3325]
 [-0.3415]
 [ 0.1585]]
Eigenvalue 3: 1.48e-15

Eigenvector 4: 
[[ 0.6104]
 [-0.3325]
 [-0.3415]
 [ 0.1585]]
Eigenvalue 4: 1.48e-15


In [244]:

for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(4,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv),
                                         eig_vals[i] * eigv,
                                         decimal=6, err_msg='', verbose=True)
print('ok')

ok


In [246]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

print(eig_pairs)

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues

print('Eigenvalues in decreasing order:\n')
for i in eig_pairs:
    print(i[0])

[(32.27195779972982, array([-0.2049+0.j, -0.3871+0.j,  0.5465+0.j,  0.7138+0.j])), (0.27756686384004503, array([-0.009 +0.j, -0.589 +0.j,  0.2543+0.j, -0.767 +0.j])), (6.613543721484293e-15, array([ 0.6104+0.j    , -0.3325+0.2066j, -0.3415+0.2491j,  0.1585-0.52j  ])), (6.613543721484293e-15, array([ 0.6104-0.j    , -0.3325-0.2066j, -0.3415-0.2491j,  0.1585+0.52j  ]))]
Eigenvalues in decreasing order:

32.27195779972982
0.27756686384004503
6.613543721484293e-15
6.613543721484293e-15
